In [1]:
import tensorflow as tf
# #指定使用那块GUP训练
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
config = tf.ConfigProto()
# 设置最大占有GPU不超过显存的70%
config.gpu_options.per_process_gpu_memory_fraction = 0.7 
# # 重点：设置动态分配GPU
config.gpu_options.allow_growth = True
# 创建session时
tf.Session(config=config) 

In [2]:
import numpy as np
from keras.datasets import cifar10
from torch.utils.data import Dataset
import torch.utils.data as Data
class MyDataset(Dataset):
    def __init__(self, imgs, transform=None):
        # super().__init__()
        self.imgs = imgs
        self.transform = transform

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, index):
        img = self.imgs[index]
        if self.transform is not None:
            img = self.transform(img)
        else:
            img = torch.from_numpy(img)
        img=img.reshape([3,32,32])
        return img
import torch
from torch.autograd import Variable
import model
import torch.nn.functional as F
model = model.cifar10(128)
model.load_state_dict(torch.load('./log/default/best-85.pth'))
model.cuda()
def EuclideanDistances(A, B):
    BT = B.transpose()
    # vecProd = A * BT
    vecProd = np.dot(A, BT)
    # print(vecProd)
    SqA = A ** 2
    # print(SqA)
    sumSqA = np.matrix(np.sum(SqA, axis=1))
    sumSqAEx = np.tile(sumSqA.transpose(), (1, vecProd.shape[1]))
    # print(sumSqAEx)

    SqB = B ** 2
    sumSqB = np.sum(SqB, axis=1)
    sumSqBEx = np.tile(sumSqB, (vecProd.shape[0], 1))
    SqED = sumSqBEx + sumSqAEx - 2 * vecProd
    SqED[SqED < 0] = 0.0
    ED = np.sqrt(SqED)
    return np.divide(ED.sum(), ED.shape[0] * ED.shape[1])


def cal_distance_image_real(images, labels):
    x_dataset = MyDataset(images)
    # print(x_dataset[0].shape)
    x_real_loader = Data.DataLoader(dataset=x_dataset, batch_size=200, shuffle=True)
    y_logits = []
    for i, data in enumerate(x_real_loader):
        # indx_target = target.clone()
        data = data.cuda()
        data = Variable(data, volatile=True)
        output = model(data)
        pred = F.softmax(output).cpu().detach().numpy()
        y_logits += [i for i in pred]
    dict = {}
    all_dis = []
    for i in range(10):
        dict[i] = []
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i] = np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)


def cal_distance_image_fake(images):
    x_dataset = MyDataset(images)
    # print(x_dataset[0].shape)
    x_real_loader = Data.DataLoader(dataset=x_dataset, batch_size=200, shuffle=True)
    y_logits = []
    labels=[]
    for i, data in enumerate(x_real_loader):
        # indx_target = target.clone()
        data = data.cuda()
        data = Variable(data, volatile=True)
        output = model(data)
        pred = output.data.max(1)[1]
        labels += [i for i in pred.cpu().numpy()]
        pred = F.softmax(output).cpu().detach().numpy()
        y_logits += [i for i in pred]

    dict = {}
    all_dis = []
    for i in range(10):
        dict[i] = []
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i] = np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)

import os
if not os.path.isdir('saved_models_{}'.format('dcgan')):
    os.mkdir('saved_models_{}'.format('dcgan'))
f = open('saved_models_{}/log_collapse1.txt'.format('dcgan'), mode='w')
import torch.utils.data as Data
import cv2

from keras.datasets import cifar10
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from PIL import Image
import matplotlib.pyplot as plt

import sys
import os
import numpy as np

class DCGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 32
        self.img_cols = 32
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100
        self.x = []
        self.y = np.zeros((31, 1), dtype=np.int)
        self.y = list(self.y)
        for i in range(31):
            self.y[i] = []

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 8 * 8, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((8, 8, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        # model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        (X_train, _), (X_test, y_test) = cifar10.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_test = (X_test.astype(np.float32) - 127.5) / 127.5

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0
        steps = []
        values = []

        for epoch in range(epochs):

            for index in range(nb_batches):
                global_step += 1
                imgs = X_train[index * batch_size:(index + 1) * batch_size]
                # labels = y_train[index * batch_size:(index + 1) * batch_size]
                # Sample noise and generate a batch of new images
                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
                gen_imgs = self.generator.predict(noise)

                # Train the discriminator (real classified as ones and generated as zeros)
                d_loss_real = self.discriminator.train_on_batch(imgs, valid)
                d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                # Train the generator (wants discriminator to mistake images as real)
                g_loss = self.combined.train_on_batch(noise, valid)
                sample_num=5000

                if global_step % save_interval == 0:
                    print("epoch:%d step:%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch,global_step, d_loss[0], 100 * d_loss[1], g_loss))

                    self.mode_drop(epoch,X_test,y_test,sample_num, global_step)


    def mode_drop(self, epoch,x_test,y_test,sample_num, global_step):
        r, c = 10, 1000
        noise = np.random.normal(0, 1, (r * c, 100))
        # sampled_labels = np.array([num for _ in range(r) for num in range(c)])
        gen_imgs = self.generator.predict([noise])
        labels = np.squeeze(y_test[:sample_num])
        labels = np.squeeze(labels)
        dis_real = cal_distance_image_real(x_test[:sample_num], labels)
        dis_fake = cal_distance_image_fake(gen_imgs)
        dis_cha = dis_real - dis_fake
        
        print('##############')
        # print(dis_real)
        # print(dis_fake)
        print(dis_cha)
        print('##########')
        f.writelines('\n')
        f.writelines('step:' + str(global_step))
        f.writelines('\n')
        f.writelines('紧度')
        f.writelines('\n')
        f.writelines(' %.8f ' % (i) for i in dis_cha)
        f.writelines('\n')
if __name__ == '__main__':
    dcgan = DCGAN()
    dcgan.train(epochs=50, batch_size=64, save_interval=200)


Using TensorFlow backend.
Sequential(
  (0): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (2): ReLU()
  (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (5): ReLU()
  (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (9): ReLU()
  (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (12): ReLU()
  (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (14): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 16, 32)        896       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 64)          18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 8, 64)          256       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 64)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 8, 8, 64)          0         
__________

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 step:200 [D loss: 0.550247, acc.: 72.66%] [G loss: 1.106247]


/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:57: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:59: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:86: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:90: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


##############
[0.88015035 0.84705945 0.81680094 0.80974238 0.77868681 0.7906159
 0.89798133 0.83300413 0.82417315 0.84928828]
##########
epoch:0 step:400 [D loss: 0.611269, acc.: 68.75%] [G loss: 1.181673]
##############
[0.84113403 0.84839224 0.80893177 0.83358759 0.83729473 0.80621141
 0.87326223 0.81462029 0.83362887 0.82429233]
##########
epoch:0 step:600 [D loss: 0.882431, acc.: 39.84%] [G loss: 0.924161]
##############
[0.84875438 0.88471881 0.81275305 0.81246224 0.79102452 0.82090392
 0.89020711 0.81522005 0.80745293 0.81522996]
##########
epoch:1 step:800 [D loss: 0.795008, acc.: 49.22%] [G loss: 0.888867]
##############
[0.86333341 0.87337321 0.81619267 0.82257824 0.7959238  0.8196529
 0.9060951  0.8095662  0.82671472 0.83322938]
##########
epoch:1 step:1000 [D loss: 0.739402, acc.: 50.78%] [G loss: 0.819836]
##############
[0.87647884 0.83945851 0.79611566 0.81833346 0.80842657 0.83386183
 0.89858219 0.81600744 0.82955747 0.84114839]
##########
epoch:1 step:1200 [D loss: 0.7

epoch:10 step:8200 [D loss: 0.739428, acc.: 42.19%] [G loss: 0.800787]
##############
[0.82954098 0.83614388 0.78581478 0.81874512 0.80226168 0.82585682
 0.8816478  0.82258364 0.81624767 0.83034763]
##########
epoch:10 step:8400 [D loss: 0.745901, acc.: 41.41%] [G loss: 0.803642]
##############
[0.83459707 0.84994118 0.79016554 0.80312147 0.8147039  0.79748811
 0.8529069  0.8024495  0.83518506 0.84671573]
##########
epoch:11 step:8600 [D loss: 0.697498, acc.: 52.34%] [G loss: 0.824998]
##############
[0.8439825  0.85471287 0.80210931 0.78924979 0.78198268 0.82464851
 0.8703156  0.83986628 0.84187066 0.84071625]
##########
epoch:11 step:8800 [D loss: 0.671702, acc.: 60.94%] [G loss: 0.886734]
##############
[0.86273235 0.85736771 0.79499343 0.80173454 0.81717208 0.83465194
 0.86340756 0.83476769 0.83556675 0.83467372]
##########
epoch:11 step:9000 [D loss: 0.647830, acc.: 65.62%] [G loss: 0.826627]
##############
[0.84512237 0.8608667  0.79471946 0.80819371 0.80192523 0.81551638
 0.8700

epoch:20 step:16000 [D loss: 0.666832, acc.: 64.06%] [G loss: 0.775941]
##############
[0.82311891 0.8513523  0.8016375  0.83083967 0.80556541 0.82719117
 0.89165449 0.81507229 0.82304898 0.83243708]
##########
epoch:20 step:16200 [D loss: 0.708075, acc.: 50.78%] [G loss: 0.798870]
##############
[0.87518188 0.8336801  0.80005443 0.81533768 0.80039243 0.79992027
 0.88112113 0.82862177 0.81722317 0.83163686]
##########
epoch:20 step:16400 [D loss: 0.660394, acc.: 60.94%] [G loss: 0.880207]
##############
[0.83807833 0.82861815 0.79340227 0.79495897 0.79413235 0.82149328
 0.89998439 0.81586292 0.82945748 0.82088937]
##########
epoch:21 step:16600 [D loss: 0.651824, acc.: 59.38%] [G loss: 0.826308]
##############
[0.87350892 0.86791599 0.78433691 0.81730249 0.80817263 0.82058567
 0.88279835 0.81651012 0.82667927 0.8222022 ]
##########
epoch:21 step:16800 [D loss: 0.682261, acc.: 59.38%] [G loss: 0.846219]
##############
[0.869607   0.86280561 0.8031876  0.80153062 0.82797122 0.80712309
 0

epoch:30 step:23800 [D loss: 0.658500, acc.: 60.94%] [G loss: 0.926948]
##############
[0.85240279 0.82823651 0.81235057 0.81196806 0.79006136 0.82097823
 0.8682491  0.83857077 0.81548991 0.81675871]
##########
epoch:30 step:24000 [D loss: 0.694968, acc.: 51.56%] [G loss: 0.821235]
##############
[0.85364902 0.84839908 0.81604474 0.8007826  0.78773399 0.82606474
 0.87172554 0.8205119  0.83432576 0.83314401]
##########
epoch:30 step:24200 [D loss: 0.643545, acc.: 60.94%] [G loss: 0.928423]
##############
[0.86264773 0.83391901 0.78821326 0.83238137 0.81825731 0.81416399
 0.89050822 0.80214722 0.83933656 0.84195561]
##########
epoch:31 step:24400 [D loss: 0.641738, acc.: 63.28%] [G loss: 0.794345]
##############
[0.85065886 0.83089206 0.79785968 0.80198679 0.78315123 0.81196021
 0.87323297 0.82381301 0.8286566  0.83867947]
##########
epoch:31 step:24600 [D loss: 0.666880, acc.: 59.38%] [G loss: 0.859963]
##############
[0.85515092 0.86422832 0.79193927 0.82581396 0.78338796 0.82619831
 0

epoch:40 step:31600 [D loss: 0.728725, acc.: 47.66%] [G loss: 0.787801]
##############
[0.86400665 0.87009165 0.79778304 0.79553781 0.78949553 0.80673314
 0.89560433 0.82603701 0.82431516 0.83703843]
##########
epoch:40 step:31800 [D loss: 0.662511, acc.: 60.16%] [G loss: 0.963825]
##############
[0.85023593 0.84610265 0.79824176 0.78549935 0.80833658 0.80504804
 0.84993232 0.80907511 0.80126017 0.83399101]
##########
epoch:40 step:32000 [D loss: 0.659641, acc.: 68.75%] [G loss: 0.846484]
##############
[0.84605979 0.83713613 0.80937511 0.81483961 0.78427408 0.81107224
 0.90684482 0.81305397 0.81754204 0.8412964 ]
##########
epoch:41 step:32200 [D loss: 0.625453, acc.: 67.97%] [G loss: 0.823433]
##############
[0.85309038 0.85098424 0.80639951 0.79847235 0.79433719 0.810828
 0.88372539 0.82138863 0.81153799 0.83324594]
##########
epoch:41 step:32400 [D loss: 0.708280, acc.: 51.56%] [G loss: 0.807787]
##############
[0.84800138 0.85350544 0.78657738 0.81236574 0.78016227 0.810899
 0.892